#### This notebook demonstrates the use of fairness metrics and an odds-equalizing post-processing algorithm for bias mitigiation.


In [1]:
%matplotlib inline
# Load all necessary packages

import sys
import numpy as np

sys.path.append("../")
from aif360.datasets import GermanDataset
from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

ImportError: No module named samya.datasets

### Fairness metrics for original dataset

In [ ]:
# import dataset
dataset_orig = GermanDataset()

#### Divide dataset into train, and test partitions (70-30)

In [ ]:
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

#### Training data characteristics

In [ ]:
# print out some labels, names, etc.
display(Markdown("#### Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Metric for the original datasets (without any classifiers)

In [ ]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

### Train classifier (logistic regression on original training data)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve

# Placeholder for predicted and transformed datasets
dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
dataset_new_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_new_test_pred = dataset_orig_test.copy(deepcopy=True)

# Logistic regression classifier and predictions for training data
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()
lmod = LogisticRegression()
lmod.fit(X_train, y_train)
fav_idx = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]
y_train_pred_prob = lmod.predict_proba(X_train)[:,fav_idx]

# Prediction probs for validation and testing data
X_test = scale_orig.transform(dataset_orig_test.features)
y_test_pred_prob = lmod.predict_proba(X_test)[:,fav_idx]

### Perform odds equalizing post processing on labels

In [ ]:
# Load post-processing algorithm that equalizes the odds
from aif360.algorithms.postprocessing.eq_odds_postprocessing import EqOddsPostprocessing
from tqdm import tqdm

# Thresholds
all_thresh = np.linspace(0.01, 0.99, 99)
display(Markdown("#### Classification thresholds used for validation and parameter selection"))

bef_avg_odds_diff_test = []
bef_avg_odds_diff_train = []
aft_avg_odds_diff_test = []
aft_avg_odds_diff_train = []
bef_bal_acc_train = []
bef_bal_acc_test = []
aft_bal_acc_train = []
aft_bal_acc_test = []
for thresh in tqdm(all_thresh):
    
    # Metrics for original training data
    y_train_pred = np.zeros_like(dataset_orig_train_pred.labels)
    y_train_pred[y_train_pred_prob >= thresh] = dataset_orig_train_pred.favorable_label
    y_train_pred[~(y_train_pred_prob >= thresh)] = dataset_orig_train_pred.unfavorable_label
    dataset_orig_train_pred.labels = y_train_pred
    
    classified_metric_orig_train = ClassificationMetric(dataset_orig_train,
                                                 dataset_orig_train_pred,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
    bef_avg_odds_diff_train.append(classified_metric_orig_train.average_odds_difference())
    bef_bal_acc_train.append(0.5*(classified_metric_orig_train.true_positive_rate()+
                              classified_metric_orig_train.true_negative_rate()))
    
    # Metrics for original validation data    
    y_test_pred = np.zeros_like(dataset_orig_test_pred.labels)
    y_test_pred[y_test_pred_prob >= thresh] = dataset_orig_test_pred.favorable_label
    y_test_pred[~(y_test_pred_prob >= thresh)] = dataset_orig_test_pred.unfavorable_label
    dataset_orig_test_pred.labels = y_test_pred
    
    classified_metric_orig_test = ClassificationMetric(dataset_orig_test,
                                                 dataset_orig_test_pred,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
    bef_avg_odds_diff_test.append(classified_metric_orig_test.average_odds_difference())
    bef_bal_acc_test.append(0.5*(classified_metric_orig_test.true_positive_rate()+
                              classified_metric_orig_test.true_negative_rate()))
    
    # Learn parameters to equalize odds and apply to create a new dataset
    pp = EqOddsPostprocessing(privileged_groups = privileged_groups,
                              unprivileged_groups = unprivileged_groups)
    pp = pp.fit(dataset_orig_train, dataset_orig_train_pred)
    dataset_new_train_pred = pp.predict(dataset_orig_train_pred)
    dataset_new_test_pred = pp.predict(dataset_orig_test_pred)
    
    # Metrics for new training data
    classified_metric_new_train = ClassificationMetric(
                                     dataset_orig_train, 
                                     dataset_new_train_pred,
                                     unprivileged_groups=unprivileged_groups,
                                     privileged_groups=privileged_groups)
    aft_avg_odds_diff_train.append(classified_metric_new_train.average_odds_difference())
    aft_bal_acc_train.append(0.5*(classified_metric_new_train.true_positive_rate()+
                              classified_metric_new_train.true_negative_rate()))

    # Metrics for new validation data
    classified_metric_new_test = ClassificationMetric(dataset_orig_test,
                                                 dataset_new_test_pred,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
    aft_avg_odds_diff_test.append(classified_metric_new_test.average_odds_difference())
    aft_bal_acc_test.append(0.5*(classified_metric_new_test.true_positive_rate()+
                                  classified_metric_new_test.true_negative_rate()))

### Plot accuracy and fairness measures for original and post-processed training data

In [ ]:
bef_bal_acc_train = np.array(bef_bal_acc_train)
bef_avg_odds_diff_train = np.array(bef_avg_odds_diff_train)

aft_bal_acc_train = np.array(aft_bal_acc_train)
aft_avg_odds_diff_train = np.array(aft_avg_odds_diff_train)

fig, ax1 = plt.subplots(figsize=(13,7))
ax1.plot(all_thresh, bef_bal_acc_train, color='b')
ax1.plot(all_thresh, aft_bal_acc_train, color='b', linestyle='dashed')
ax1.set_title('Original and Postprocessed training data', fontsize=16, fontweight='bold')
ax1.set_xlabel('Classification Thresholds', fontsize=16, fontweight='bold')
ax1.set_ylabel('Balanced Accuracy', color='b', fontsize=16, fontweight='bold')
ax1.xaxis.set_tick_params(labelsize=14)
ax1.yaxis.set_tick_params(labelsize=14)

ax2 = ax1.twinx()
ax2.plot(all_thresh, np.abs(bef_avg_odds_diff_train), color='r')
ax2.plot(all_thresh, np.abs(aft_avg_odds_diff_train), color='r', linestyle='dashed')
ax2.set_ylabel('abs(averaged odds diff)', color='r', fontsize=16, fontweight='bold')
ax2.yaxis.set_tick_params(labelsize=14)
ax2.grid(True)
fig.legend(["Balanced Acc. - Orig.", "Balanced Acc. - Postproc.",
             "Odds diff. - Orig.","Odds diff. - Postproc.",], 
           fontsize=16)

### Plot accuracy and fairness measures for original and post-processed testing data

In [ ]:
bef_bal_acc_test = np.array(bef_bal_acc_test)
bef_avg_odds_diff_test = np.array(bef_avg_odds_diff_test)

aft_bal_acc_test = np.array(aft_bal_acc_test)
aft_avg_odds_diff_test = np.array(aft_avg_odds_diff_test)

fig, ax1 = plt.subplots(figsize=(13,7))
ax1.plot(all_thresh, bef_bal_acc_test, color='b')
ax1.plot(all_thresh, aft_bal_acc_test, color='b', linestyle='dashed')
ax1.set_title('Original and Postprocessed testing data', fontsize=16, fontweight='bold')
ax1.set_xlabel('Classification Thresholds', fontsize=16, fontweight='bold')
ax1.set_ylabel('Balanced Accuracy', color='b', fontsize=16, fontweight='bold')
ax1.xaxis.set_tick_params(labelsize=14)
ax1.yaxis.set_tick_params(labelsize=14)


ax2 = ax1.twinx()
ax2.plot(all_thresh, np.abs(bef_avg_odds_diff_test), color='r')
ax2.plot(all_thresh, np.abs(aft_avg_odds_diff_test), color='r', linestyle='dashed')
ax2.set_ylabel('abs(averaged odds diff)', color='r', fontsize=16, fontweight='bold')
ax2.yaxis.set_tick_params(labelsize=14)
ax2.grid(True)
fig.legend(["Balanced Acc. - Orig.", "Balanced Acc. - Postproc.",
            "Odds diff. - Orig.", "Odds diff. - Postproc."], 
           fontsize=16)